[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/crewai/crewai-query-agent-as-tool.ipynb)

## Weaviate Query Agent with Crew AI

This notebook will show you how to define the Weaviate Query Agent as a tool through the Crew AI.

### Requirements
1. Weaviate Cloud instance (WCD): The Weaviate Query Agent is only accessible through WCD at the moment. You can create a serverless cluster or a free 14-day sandbox [here](https://console.weaviate.cloud/).
2. Install Crew AI with `pip install crewai`
3. Install the Weaviate Agents package with `pip install weaviate-agents`
4. You'll need a Weaviate cluster with data. If you don't have one, check out [this notebook](integrations/Weaviate-Import-Example.ipynb) to import the Weaviate Blogs.


### Import libraries and keys

In [1]:
import weaviate
from weaviate_agents.query import QueryAgent
import os

from crewai.tools import tool
from crewai import Agent
from crewai import Task
from crewai import Crew, Process
from pydantic import BaseModel, Field
from typing import Type
from crewai.tools import BaseTool


/usr/local/lib/python3.11/site-packages/litellm/utils.py:149: DeprecationWarning: open_text is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with resources.open_text(


In [2]:
os.environ["WEAVIATE_URL"] = ""
os.environ["WEAVIATE_API_KEY"] = ""

### Define Weaviate Query Agent as a tool

In [3]:
class WeaviateQuerySchema(BaseModel):
    """Input for WeaviateQueryAgentTool."""

    query: str = Field(
        ...,
        description="The query to search retrieve relevant information from the Weaviate database. Pass only the query, not the question.",
    )


class WeaviateQueryAgentTool(BaseTool):
    name: str = Field(default="Weaviate Query Agent")
    description: str = Field(
        default="Send a query to the database and get the response."
    )
    args_schema: Type[BaseModel] = WeaviateQuerySchema

    def send_query_agent_request(self, query: str) -> str:
        """
        Send a query to the database and get the response.

        Args:
            query (str): The question or query to search for in the database. This can be any natural language question related to the content stored in the database.

        Returns:
            str: The response from the database containing relevant information.
        """

        weaviate_client = weaviate.connect_to_weaviate_cloud(
            cluster_url=os.getenv("WEAVIATE_URL"),
            auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
        )
        query_agent = QueryAgent(
            client=weaviate_client,
            collections=[
                "Blogs" # we are using the Weaviate Embeddings for our Blogs collection
            ],
        )
        runner = query_agent.run(query)
        print("runner", runner)
        return runner.final_answer

    def _run(self, query: str) -> str:
        return self.send_query_agent_request(query)
    

query_agent_tool = WeaviateQueryAgentTool()

### 

In [5]:
researcher = Agent(
    role="Blog Content Researcher",
    goal="Find relevant blog posts and extract key information",
    backstory="You're specialized in analyzing blog content to extract insights and answers",
    verbose=True,
    tools=[query_agent_tool]
)

In [6]:
research_task = Task(
    description="Research blog posts about packaging software applications with Docker",
    expected_output="A summary of key information from relevant blog posts",
    agent=researcher
)

In [7]:
blog_crew = Crew(
    agents=[researcher],
    tasks=[research_task],
    process=Process.sequential,
    verbose=True
)

### Query Time

In [8]:
result = blog_crew.kickoff()

print(result)

# Agent: Blog Content Researcher
## Task: Research blog posts about packaging software applications with Docker
runner original_query='packaging software applications with Docker' collection_names=['Blogs'] searches=[[QueryResultWithCollection(queries=['packaging software applications with Docker'], filters=[[]], filter_operators='AND', collection='Blogs')]] aggregations=[] usage=Usage(requests=3, request_tokens=6692, response_tokens=386, total_tokens=7078, details=None) total_time=10.477295398712158 aggregation_answer=None has_aggregation_answer=False has_search_answer=True is_partial_answer=False missing_information=[] final_answer="Docker is a platform that uses OS-level virtualization to package software applications in units called containers. These containers are similar to lightweight virtual machines, possessing their own file systems and operating system libraries, yet sharing the host system's kernel. Containers are beneficial for software application packaging as they provid

/usr/local/lib/python3.11/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(




# Agent: Blog Content Researcher
## Final Answer: 
Docker is a platform that uses OS-level virtualization to package software applications in units called containers. These containers are similar to lightweight virtual machines, possessing their own file systems and operating system libraries, yet sharing the host system's kernel. Containers are beneficial for software application packaging as they provide considerable isolation with reduced overhead compared to traditional virtual machines. 

A standard Docker practice is to package a single application per container, with the container's lifecycle managed by the application's main process. If this process ends, the container typically stops. This approach ensures applications run in isolation with a consistent environment across different systems.

One of the key benefits of using Docker is its portability: as long as the Docker Engine is installed, containers can run on any OS. Docker also aids in maintaining isolation and predict